In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\ravis\\OneDrive\\Desktop\\Text-Summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

# entity - a class that is used to store the configuration of the project
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading file from: {self.config.source_URL}")
            filename, _ = request.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f"Downloaded file: {filename} with size: {get_size(Path(filename))}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file} with size: {get_size(self.config.local_data_file)}")
    

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            logger.info(f"Extracting file to: {unzip_path}")
            zip_ref.extractall(unzip_path)
            logger.info(f"Extraction completed. Files extracted to: {unzip_path}")


In [9]:
import os
print(os.getcwd())

c:\Users\ravis\OneDrive\Desktop\Text-Summarization


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e

[2025-06-12 22:34:53,624: INFO common yaml file: config\config.yaml loaded successfully]
[2025-06-12 22:34:53,628: INFO common yaml file: params.yaml loaded successfully]
[2025-06-12 22:34:53,629: INFO common created directory at: artifacts]
[2025-06-12 22:34:53,631: INFO common created directory at: artifacts/data_ingestion]
[2025-06-12 22:34:53,633: INFO 767424506 File already exists: artifacts\data_ingestion\data.zip with size: ~ 7718 KB]
[2025-06-12 22:34:53,640: INFO 767424506 Extracting file to: artifacts\data_ingestion]
[2025-06-12 22:34:53,796: INFO 767424506 Extraction completed. Files extracted to: artifacts\data_ingestion]
